# Sabah Rental Analysis

look at this website for reference

https://adtarie.net/posts/005-webscraping-machinelearning-rent-pricing/

In [1]:
#importing libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('max_colwidth', 200)
from bs4 import BeautifulSoup as bs
import requests
import re
import time
import datetime
import smtplib
import json
from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations

!jupyter nbextension enable --py widgetsnbextension #enabling progress bar

Please specify one nbextension/package at a time


page_number function is to generate a list of webpage address for a given page number and when executed, returns a list of listing web address 

In [2]:
def page_number(start,end):         
    page_url = 'https://www.mudah.my/kedah/properties-for-rent?o=' #tinggal Terengganu,Kedah,Kelantan,Melaka,N9,Pahang,Perak,Perlis
    list_page = []
    for i in range(start, end+1):
        list_page.append(page_url+str(i))
    return list_page

page_number(2,4)

['https://www.mudah.my/kedah/properties-for-rent?o=2',
 'https://www.mudah.my/kedah/properties-for-rent?o=3',
 'https://www.mudah.my/kedah/properties-for-rent?o=4']

get_list_html will setting up a list of page in a given url and return a list of listing ads

In [3]:
def get_list_html(page_url): 
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0"} #google my user agent
    page = requests.get(url=page_url, headers=headers)
    soup = bs(page.text, "html.parser")
    
    script_tag = soup.find('script', type='application/ld+json')
    if script_tag is not None:
        data = json.loads(script_tag.text)
        dict_query = data[2]['itemListElement']
        
        n_query = data[2]['numberOfItems']
        list_html = []
        
        for i in range(n_query):
            link = data[2]['itemListElement'][i]['item']['url']
            list_html.append(link)
        
        return list_html
    else:
        return []  

print(get_list_html('https://www.mudah.my/kedah/properties-for-rent?o=2')[1:5])


['https://www.mudah.my/Rumah+untuk+disewa+taman+langkawi+kuah-106116687.htm', 'https://www.mudah.my/Kedai+2+Tingkat+untuk+Sewa+di+Lunas-105845471.htm', 'https://www.mudah.my/Rumah+Berkembar+Setingkat+untuk+disewa+di+Lunas-105857541.htm', 'https://www.mudah.my/Ground+Floor+Shoplot+For+Rent-106116221.htm']


get_list_url generate listing property from each page of n_page

In [4]:
def get_list_url(n_page):
    list_html = []
    for i in tqdm(range(n_page)):
        list_html.extend(get_list_html(page_number(1,n_page)[i]))
    return list_html
get_list_url(1)[:3]   

  0%|          | 0/1 [00:00<?, ?it/s]

['https://www.mudah.my/Taman+Ria+Jaya+Terrace+House+for+Rent-105335835.htm',
 'https://www.mudah.my/+FULLY+GATED+GUARDED+PERMAIPURA+BEDONG+GURUN+2+Stry+Terrace+for+RENT-106124181.htm',
 'https://www.mudah.my/3+Storey+Shop+Lot+Bakar+Arang-103475686.htm']

get_list_dict extract data from url and its function is to get dataset (atribut of units) in a form of dictionary and returns a dictionaries of attributes inside a list

In [5]:
def get_list_dict(n_page):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0"}
    list_html = get_list_url(n_page)
    list_dict = []
    
    for url in tqdm(list_html):
        dict_unit = {}  # Initialize with a default value
        try:
            page = requests.get(url=url, headers=headers)
            soup1 = bs(page.text, "html.parser")
            soup2 = bs(soup1.prettify(), 'html.parser')

            id_html = re.search(r'(\d+).htm', url).group(1)
            title = soup2.find(itemprop='name').text.strip()

            script_tag = soup2.find("script", id="__NEXT_DATA__")
            script_content = script_tag.text
            data = json.loads(script_content)
            props = data.get("props", {})
            id_listing = re.search(r'-(\d+)\.htm', url).group(1)

            dict_id = [{'realValue': '', 'id': 'ads_id', 'value': id_listing, 'label': 'id ads'}]
            dict_building = props["initialState"]["adDetails"]["byID"][id_listing]["attributes"]['propertyParams'][2]['params']
            dict_prop = props["initialState"]["adDetails"]["byID"][id_listing]["attributes"]['categoryParams']
            dict_unit = dict_id + dict_building + dict_prop
        except:
            pass  # You can put a specific action here if needed
        
        list_dict.append(dict_unit)

    return list_dict

# Sanity check
get_list_dict(1)[4]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

{}

In [6]:
#getting values out of dictionary
def get_values(list_dict):
    """
    Description:
        Function to values of the previous dictionary.

    Parameters:
        list_dict (list): list of dictionary where attributes stored
        
    Returns:
        a list of values (unit/property) attributes
    
    """
    keys = [
        'ads_id',
        'prop_name',
        # 'developer_name', 
        # 'address', 
        'completion_year', 
        # 'num_floors', 
        # 'num_units',
        'monthly_rent', 
        # 'category_id', 
        'location', 
        'property_type', 
        # 'floor_range', 
        'rooms', 
        'parking',
        'bathroom', 
        'size', 
        'furnished',
        'facilities', 
        'additional_facilities', 
       ]

    values = {}
    for key in keys:
        try:
            values[key] = next(item['value'] for item in list_dict if item["id"] == key)
        except StopIteration:
            values[key] = None
    return values

#sanity check
get_values(get_list_dict(1)[5])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

{'ads_id': None,
 'prop_name': None,
 'completion_year': None,
 'monthly_rent': None,
 'location': None,
 'property_type': None,
 'rooms': None,
 'parking': None,
 'bathroom': None,
 'size': None,
 'furnished': None,
 'facilities': None,
 'additional_facilities': None}

get_df_final will generate a dataframe from the list

In [24]:
#get df from list
def get_df_final(n_page):
    
    list_data = get_list_dict(n_page)
    list_new = []
    for i in range(0,len(list_data)):
            dic = get_values(list_data[i])
            list_new.append(dic)
    
    df = pd.DataFrame(list_new)
    return df

Before we scrape, lets test with the first page and call the first the unit with the dataframe transpose

In [16]:
#sanity check
#scrape how many pages, 
get_df_final(1).head(3).T

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

,0,1,2
ads_id,None,105245462,105231263
prop_name,None,Urbaneeze Apartment,Nu Ryegates
completion_year,None,2016,2020
monthly_rent,None,"RM 1,800 per month","RM 1,800 per month"
location,None,Sarawak - Kuching,Sarawak - Kuching
property_type,None,Apartment,Apartment
rooms,None,3,3
parking,None,1,2
bathroom,None,2,2
size,None,1399 sq.ft.,1150 sq.ft.


In [25]:
### save to file
df_= get_df_final(3) #page 20 can
#df_.to_csv('SarawakMudahRent100pg.csv', index=False)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/90 [00:00<?, ?it/s]

In [26]:
df_.to_csv('PerlisMudahRent3pg.csv', index=False)